<a href="https://colab.research.google.com/github/joseantbelloac/Talentpitch/blob/main/Talentpitch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: instalar pyspark

!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=2453c9337ce943a45a4112d991ed365ff62e26453787e7e1d93096db96d813e0
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [2]:
# prompt: crear una carpeta para subir archivos csv que se llame "taletpitch"

!mkdir talentpitch


In [12]:
import pandas as pd

#cleaning process

df_chall=pd.read_csv("talentpitch/challenges - challenges.csv", sep=',')
df_prof=pd.read_csv("talentpitch/profiles - Hoja 1.csv")
df_resum=pd.read_csv("talentpitch/resumes - resumes.csv")
df_user=pd.read_csv("talentpitch/users - users.csv")

In [17]:
df_resum

,id,user_id,name,type,video,views,created_at
0,1,300,mi primer video,pitch_video,https://media.talentpitch.co/resumes/1/akjdaks...,120,2024-06-06
1,2,400,personal pitch,pitch_video,https://media.talentpitch.co/resumes/2/dadkajd...,1,2024-06-07
2,3,500,soy,pitch_video,https://media.talentpitch.co/resumes/3/1231283...,2,2024-06-08
3,4,600,hola a todos,pitch_video,https://media.talentpitch.co/resumes/4/aws_kaj...,4,2024-06-09
4,5,700,gran talento,pitch_video,https://media.talentpitch.co/resumes/5/aws_123...,55,2024-06-10
5,6,800,mi primer video,pitch_video,https://media.talentpitch.co/resumes/6/temp_aw...,56,2024-06-11
6,7,900,cantando un poco,pitch_video,https://media.talentpitch.co/resumes/7/2213231...,234,2024-06-12
7,8,1000,conoceme,pitch_video,https://media.talentpitch.co/resumes/8/2213231...,123,2024-06-13
8,9,1000,experiencia profesional,resume,https://media.talentpitch.co/resumes/9/ahdgadh...,123,2024-06-14
9,10,2000,hola soy apasionado por la tecnología,pitch_video,https://media.talentpitch.co/resumes/10/temp_t...,77,2024-06-15


In [14]:
# prompt: leer archivo challenges sin usar pandas y reemplazar " por ,

def replace_quotes_with_commas(file_path):
  with open(file_path, 'r') as f:
    lines = f.readlines()

  new_lines = []
  for line in lines:
    new_line = line.replace('"', ',')
    new_lines.append(new_line)

  return new_lines

file_path = "talentpitch/challenges - challenges.csv"
new_lines = replace_quotes_with_commas(file_path)

# You can now process the new_lines list as needed. For example, you can print it:
for line in new_lines:
    print(line)


id,name,description,status,opencall_objective,created_at

,450,Cantantes de Reguetonland que buscan patrocinios,,,,

,Buscamos jóvenes comprometidos con la justicia social, el cambio climático y los derechos humanos, listos para desafiar el sistema. Ofrecemos oportunidades de conexión con entidades políticas progresistas, ONGs y celebridades comprometidas con el cambio social.,,,published,Artistas,2024-07-05,

460,Conviértete en el próximo líder del cambio social.,¿Te gustaría convertirte en un compositor de canciones y aprender de expertos en la industria?,published,Artistas,2024-07-06

470,Te gustaría convertirte en un compositor,Formación personalizada y la oportunidad de aprender de los mejores en la industria de la televisión. ¡Es tu chance para mejorar tus habilidades y destacar como presentador de TV!,published,Artistas,2024-07-07

480,Sé el mejor presentador de televisión,¡Domina el arte del casting con una mentoría exclusiva! ,open,Artistas,2024-07-08

490,Domina el arte del c

In [4]:
from pyspark.sql import SparkSession

# Step 3: Create a Spark Session
spark = SparkSession.builder.appName("CSV to Spark Database").config("spark.sql.catalogImplementation", "hive").enableHiveSupport().getOrCreate()

# Step 4: Load CSV files into DataFrames
df1 = spark.read.csv("talentpitch/challenges - challenges.csv", header=True, inferSchema=True)
df2 = spark.read.csv("talentpitch/profiles - Hoja 1.csv", header=True, inferSchema=True)
df3 = spark.read.csv("talentpitch/resumes - resumes.csv", header=True, inferSchema=True)
df4 = spark.read.csv("talentpitch/users - users.csv", header=True, inferSchema=True)


df1.createOrReplaceTempView("challenges")
df2.createOrReplaceTempView("profiles")
df3.createOrReplaceTempView("resumes")
df4.createOrReplaceTempView("users")

In [21]:
query1=spark.sql(""" SELECT
id
,name
,description
,status
,opencall_objective
,created_at
FROM challenges
where 1=1
and year(created_at)*100+month(created_at)>=year(current_date())*100+month(current_date())-3
and status='open'
order by created_at desc
--limit 10
""")

query1.show()

+---+--------------------+--------------------+------+------------------+----------+
| id|                name|         description|status|opencall_objective|created_at|
+---+--------------------+--------------------+------+------------------+----------+
|490|Domina el arte de...|¿Sueñas con conve...|  open|         Innovador|2024-07-09|
|480|Sé el mejor prese...|¡Domina el arte d...|  open|          Artistas|2024-07-08|
+---+--------------------+--------------------+------+------------------+----------+



In [32]:
from pyspark.sql.functions import add_months, current_date

query2=spark.sql("""
select
p.user_id
,'be_discover' as main_objective
,r.video
,p.updated_at
,max(r.created_at) as created_at
from profiles p
inner join resumes r
on p.user_id=r.user_id
where 1=1
and p.onboarding_goal like '%be_discovered%'
AND p.created_at >= add_months(current_date(), -90)
and r.type='pitch_video'
and r.video is not null
group by
p.user_id
,r.video
,p.updated_at
--order by p.created_at desc

""")
query2.show()


+-------+--------------+--------------------+----------+----------+
|user_id|main_objective|               video|updated_at|created_at|
+-------+--------------+--------------------+----------+----------+
|    900|   be_discover|https://media.tal...|2024-04-15|2024-06-12|
|   2000|   be_discover|https://media.tal...|2024-04-20|2024-06-15|
|    600|   be_discover|https://media.tal...|2024-04-08|2024-06-09|
|    700|   be_discover|https://media.tal...|2024-04-09|2024-06-10|
|    500|   be_discover|https://media.tal...|2024-04-07|2024-06-08|
|    800|   be_discover|https://media.tal...|2024-04-22|2024-06-11|
|    400|   be_discover|https://media.tal...|2024-04-06|2024-06-07|
|   3000|   be_discover|https://media.tal...|2024-04-19|2024-06-17|
+-------+--------------+--------------------+----------+----------+



In [36]:
query3=spark.sql("""
select
p.user_id
,u.slug
,p.views
from profiles p
inner join users u
on p.user_id=u.id
where 1=1
order by p.views desc
limit 10

""")
query3.show()

+-------+----------------+-----+
|user_id|            slug|views|
+-------+----------------+-----+
|    900|    anna-beltran|  200|
|    800|  juanjo-munoz-2|  190|
|   3000|      clara-bena|   70|
|   1000|luisita-perez-22|   59|
|    600|   pedro-jime-23|   40|
|    500|         jj-diaz|   30|
|    700|        lis-duar|   30|
|    400| juanjo-suarez-2|   20|
|   2000|     jj-bbarreto|    1|
+-------+----------------+-----+

